# Workbook 2 Selecting Features
For this workbook, I wanted to select my own features of what is most important when considering a houses value and compare it to how the computer will select based on Lasso (seperate workbook)

In [30]:
import numpy as np
import pandas as pd
import patsy
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.model_selection import cross_val_score
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.feature_selection import SelectKBest

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [31]:
train = pd.read_csv('./train.csv')

In [32]:
test = pd.read_csv('./test.csv')

In [33]:
y = train['SalePrice']
X = train.drop(['SalePrice'], axis = "columns")

In [34]:
test_id = test['Id']

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

In [36]:
def clean_me(df):
    #Replaces Null values outside of object columns 
    for i in range(len(df.columns)): 
        if [df.iloc[: ,i].dtypes!=object] == True:
            mean = df.iloc[: ,i].mean()
            df.iloc[: ,i].fillna(mean, inplace=True)
        else:
            pass
        #Replaces other null values with AANone
    str_cols = df.columns[df.dtypes==object]
    df[str_cols] = df[str_cols].fillna('AANone')
    df.fillna(0,inplace=True)
        #Change selected columns from Int to float
    for col in ['Low Qual Fin SF', 'Gr Liv Area', 'Lot Area', '1st Flr SF', '2nd Flr SF',
                'Wood Deck SF', 'Open Porch SF', 'Enclosed Porch', '3Ssn Porch', 'Screen Porch', 'Pool Area', 'Misc Val']:
                df[col] = df[col].astype('float64')
        #Makes remaining int colomns objects 
    int_cols = df.columns[df.dtypes=='int64']
    df[int_cols] = df[int_cols].astype('object')
    df = df.drop(['Id', 'PID'], axis=1)
    return df

In [37]:
X_train = clean_me(X_train)

C:\Users\jmvail\Anaconda3\envs\dsi\lib\site-packages\pandas\core\frame.py:2540: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]
C:\Users\jmvail\Anaconda3\envs\dsi\lib\site-packages\pandas\core\frame.py:3035: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)
C:\Users\jmvail\Anaconda3\envs\dsi\lib\site-packages\ipykernel\__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-doc

In [38]:
X_test = clean_me(X_test)

C:\Users\jmvail\Anaconda3\envs\dsi\lib\site-packages\pandas\core\frame.py:2540: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]
C:\Users\jmvail\Anaconda3\envs\dsi\lib\site-packages\pandas\core\frame.py:3035: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)
C:\Users\jmvail\Anaconda3\envs\dsi\lib\site-packages\ipykernel\__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-doc

In [39]:
test = clean_me(test)

**Cleaning Done**

Chose variables (dummies chosen by which I liked)

In [40]:
X_train.head()

,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,...,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type
1554,20,RL,102.0,9373.0,Pave,AANone,IR1,Lvl,AllPub,Corner,...,0.0,0.0,0.0,AANone,AANone,AANone,0.0,5,2008,WD
1222,20,RM,70.0,12702.0,Pave,AANone,Reg,Lvl,AllPub,Inside,...,0.0,0.0,0.0,AANone,AANone,AANone,0.0,12,2008,WD
1575,20,RL,67.0,8877.0,Pave,AANone,Reg,Lvl,AllPub,Inside,...,0.0,0.0,0.0,AANone,AANone,AANone,0.0,4,2006,COD
697,160,RL,24.0,2308.0,Pave,AANone,Reg,Lvl,AllPub,Inside,...,0.0,0.0,0.0,AANone,AANone,AANone,0.0,6,2009,WD
207,20,RL,85.0,15300.0,Pave,AANone,Reg,Bnk,AllPub,Inside,...,0.0,0.0,0.0,AANone,AANone,AANone,0.0,6,2010,WD


In [41]:
X_train.columns

Index(['MS SubClass', 'MS Zoning', 'Lot Frontage', 'Lot Area', 'Street',
       'Alley', 'Lot Shape', 'Land Contour', 'Utilities', 'Lot Config',
       'Land Slope', 'Neighborhood', 'Condition 1', 'Condition 2', 'Bldg Type',
       'House Style', 'Overall Qual', 'Overall Cond', 'Year Built',
       'Year Remod/Add', 'Roof Style', 'Roof Matl', 'Exterior 1st',
       'Exterior 2nd', 'Mas Vnr Type', 'Mas Vnr Area', 'Exter Qual',
       'Exter Cond', 'Foundation', 'Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure',
       'BsmtFin Type 1', 'BsmtFin SF 1', 'BsmtFin Type 2', 'BsmtFin SF 2',
       'Bsmt Unf SF', 'Total Bsmt SF', 'Heating', 'Heating QC', 'Central Air',
       'Electrical', '1st Flr SF', '2nd Flr SF', 'Low Qual Fin SF',
       'Gr Liv Area', 'Bsmt Full Bath', 'Bsmt Half Bath', 'Full Bath',
       'Half Bath', 'Bedroom AbvGr', 'Kitchen AbvGr', 'Kitchen Qual',
       'TotRms AbvGrd', 'Functional', 'Fireplaces', 'Fireplace Qu',
       'Garage Type', 'Garage Yr Blt', 'Garage Finish', 'Gara

In [42]:
col = ['Total Bsmt SF', '1st Flr SF', 'Gr Liv Area', 'Garage Area', 'MS Zoning', 'Neighborhood',
       'Condition 1', 'House Style', 'Misc Feature', 'Fireplace Qu', 'Year Remod/Add', 'Paved Drive', ]

In [43]:
X_train = X_train[col]

In [44]:
X_test = X_test[col]

In [45]:
test = test[col]

# Dummy Function
Used to make colums for dummied variables even if they are not in the training or testing data. This is so that the tables will match up in fitting and scoring

In [46]:
def get_dummied(train, test):
    list = []
    
    for i in train.columns:
        if train[i].dtype == object:
            list.append(i)
            
    full_data = pd.concat([train, test], axis=0)
    full_data = pd.get_dummies(full_data, columns=list, drop_first=True)
    
    X_dummied = full_data[:len(train)]
    test_dummied = full_data.iloc[-len(test):]

    
    return X_dummied, test_dummied

In [47]:
X_train_dum, X_test_dum = get_dummied(X_train, X_test)

In [48]:
lr = LinearRegression()
lr.fit(X_train_dum, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [49]:
print(lr.score(X_train_dum, y_train))
print(lr.score(X_test_dum, y_test))

0.8778758609085109
0.7066181675815434


Rsquared improved from first Linear Regression

In [50]:
X_train_dum, test_dum = get_dummied(X_train, test)

In [51]:
model = lr.fit(X_train_dum, y_train)
y_hat = model.predict(test_dum)

In [52]:
y_hat

array([153945.16445953, 221167.67432088, 186451.97279282, 121422.72702286,
       185532.02862181,  97041.56228182, 121952.75569975, 144659.81265315,
       198714.52976028, 161246.01076403, 160174.68664298, 116780.01636516,
       187870.47331935, 272208.94165638, 177123.312175  , 109588.00922613,
       156212.04678866, 130002.35384697, 217288.0078539 , 129104.73673699,
       150762.38839105, 115080.54261507, 221842.09031437, 159827.80191808,
       194288.93640352, 121562.76026206, 181087.2113503 ,  96316.84727572,
       133083.54736787,  39984.65889182, 111438.36924169, 117193.82926456,
       252043.56435856, 176568.5718544 , 224416.25799654, 174711.92996337,
       129275.02070079,  86217.12315733,  87792.21511835, 211546.48412681,
       204079.39495128, 206359.94773936, 175633.11363195, 170495.12737198,
       205852.89516337, 161858.18135985, 217350.34890415, 110784.17014783,
       127007.34464637, 117505.2994113 ,  93780.7518593 , 167650.88044691,
       272086.89761767, 1

In [53]:
pd.options.display.max_rows = 999

In [54]:
pd.options.display.max_rows

999

Most important coefficients based on model

In [59]:
coef = pd.DataFrame(X_train_dum.columns, model.coef_, columns=['Feature'])
coef['abs'] = abs(coef.index)
coef['coef']= coef.index
coef = coef.sort_values(['abs'], ascending=False).head(25)
coef

,Feature,abs,coef
231516.557616,House Style_2.5Fin,231516.557616,231516.557616
139074.838003,Neighborhood_GrnHill,139074.838003,139074.838003
76435.153919,Neighborhood_StoneBr,76435.153919,76435.153919
57770.835123,Neighborhood_NridgHt,57770.835123,57770.835123
57101.543308,Neighborhood_NoRidge,57101.543308,57101.543308
54643.118143,Year Remod/Add_2010,54643.118143,54643.118143
54420.650083,Year Remod/Add_2009,54420.650083,54420.650083
54347.593029,Year Remod/Add_2008,54347.593029,54347.593029
53333.811419,Neighborhood_Veenker,53333.811419,53333.811419
44853.828885,Condition 1_PosA,44853.828885,44853.828885


In [56]:
#dum_trial_2 = pd.DataFrame(y_hat, index = test_id)
#dum_trial_2.columns = ['SalePrice']
#dum_trial_2.head

In [57]:
#dum_trial_2.describe()

In [58]:
#dum_trial_2.to_csv("./dum_trial_2.csv")